In [8]:
import argparse
import pprint

import torch
from torch import optim
import torch.nn as nn

'''i have no idea where this fuck came from'''
# import torch_optimizer as custom_optim

'i have no idea where this fuck came from'

# 1. train.py

In [149]:
p = argparse.ArgumentParser()
args = p.parse_args("")


args.is_continue = False
args.model_fn = '.'
args.train = '/Users/rainism/desktop/grad/torch_study/transformer/data/tt.corpus.shuf.train.tok.bpe'
args.valid = '/Users/rainism/desktop/grad/torch_study/transformer/data/tt.corpus.shuf.train.tok.bpe'
args.lang = 'enko'
args.gpu_id = -1
args.store_true = False
args.batch_size = 160
args.n_epochs = 30
args.verbose = 1
args.init_epoch = 1
args.max_length = 100
args.dropout = .2
args.word_vec_size = 512 # word vec size
args.hidden_size = 768
args.n_layers = 4
args.max_grad_norm = 5. # 'Threshold for gradient clipping. Default=%(default)s'
args.iteration_per_update = 1 # 'Number of feed-forward iterations for one parameter update. Default=%(default)s
args.lr = 1.
args.lr_step = 1
args.lr_gamma = .5
args.lr_decay_start = 10
args.use_adam = True
args.rl_lr = .01
args.rl_n_samples = 1
args.rl_n_epochs = 10
args.rl_n_gram = 6
args.rl_reward = 'gleu'
args.use_transformer = False
args.n_splits = 8
args.off_autocast = False

config = args

# 2. data_loader.py

In [72]:
import os
import torchtext
version = list(map(int, torchtext.__version__.split('.')))
if version[0] <= 0 and version[1] < 9:
    from torchtext import data, datasets
else:
    from torchtext.legacy import data, datasets

PAD, BOS, EOS = 1, 2, 3
# PAD의 번호는 1, BOS는 2, EOS는 3인가보네


class DataLoader():

    def __init__(self,
                 train_fn=None,
                 valid_fn=None,
                 exts=None,
                 batch_size=64,
                 device='cpu',
                 max_vocab=99999999,
                 max_length=255,
                 fix_length=None,
                 use_bos=True,
                 use_eos=True,
                 shuffle=True,
                 dsl=False
                 ):

        super(DataLoader, self).__init__() # ??? 상속받을게 없는데?

        # Field -> fields -> TabularDataset -> build_vocab -> Bucket

        # src와 tgt가 각각 있는 이유는, 파일이 각각 있었기 때문이다.
            # torchtext.data.Field
        self.src = data.Field(
            sequential=True,
            use_vocab=True,
            batch_first=True,
            include_lengths=True,
            fix_length=fix_length, # None
            init_token='<BOS>' if dsl else None, # dsl : dure learning할때 필요한것. 지금은 None이라고 보면 됨.
            eos_token='<EOS>' if dsl else None,
        )

        self.tgt = data.Field(
            sequential=True,
            use_vocab=True,
            batch_first=True,
            include_lengths=True,
            fix_length=fix_length,
            init_token='<BOS>' if use_bos else None, # True .. learning에서는 필요 없고, 생성자 할때만 필요함(?)
            eos_token='<EOS>' if use_eos else None,
        )

        if train_fn is not None and valid_fn is not None and exts is not None:
            # TranslationDataset는 밑에 정의 되어있습니다.
            train = TranslationDataset(
                path=train_fn, # train file path
                exts=exts, # en,ko path가 튜플로 들어가 있음.
                fields=[('src', self.src), ('tgt', self.tgt)], # 사용할 필드 명
                max_length=max_length
            )
            valid = TranslationDataset(
                path=valid_fn,
                exts=exts,
                fields=[('src', self.src), ('tgt', self.tgt)],
                max_length=max_length,
            )

            # bucketIterator가 하는 일을 실제 데이터를 가지고 와서. -> pad까지 체운 형태로 만들고
            # 미니배치 단위로 만들어주는 역할을 한다.
            # https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.BucketIterator
            self.train_iter = data.BucketIterator(
                train,
                batch_size=batch_size,
                device='cuda:%d' % device if device >= 0 else 'cpu',
                shuffle=shuffle,
                sort_key=lambda x: len(x.tgt) + (max_length * len(x.src)), # ?????????????? what's x?
                sort_within_batch=True,
            )

            self.valid_iter = data.BucketIterator(
                valid,
                batch_size=batch_size,
                device='cuda:%d' % device if device >= 0 else 'cpu',
                shuffle=False,
                sort_key=lambda x: len(x.tgt) + (max_length * len(x.src)),
                sort_within_batch=True,
            )

            self.src.build_vocab(train, max_size=max_vocab)
            self.tgt.build_vocab(train, max_size=max_vocab)

    def load_vocab(self, src_vocab, tgt_vocab):
        self.src.vocab = src_vocab
        self.tgt.vocab = tgt_vocab


# torchtext에는 maxlen을 잘라주는게 없어서 customizing했어.
class TranslationDataset(data.Dataset):
    """Defines a dataset for machine translation."""

    @staticmethod
    def sort_key(ex):
        return data.interleave_keys(len(ex.src), len(ex.trg))

    def __init__(self, path, exts, fields, max_length=None, **kwargs):
        """Create a TranslationDataset given paths and fields.

        Arguments:
            path: Common prefix of paths to the data files for both languages.
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            Remaining keyword arguments: Passed to the constructor of
                data.Dataset.
        """
        if not isinstance(fields[0], (tuple, list)):
            # fields가 [('src',src),('tgt',tgt)]형태가 아닐때 다시 정의를 함.
            fields = [('src', fields[0]), ('trg', fields[1])]

        if not path.endswith('.'):
            # 주소의 끝에 .이 없다면 추가로 넣어줘.
            path += '.'

        src_path, trg_path = tuple(os.path.expanduser(path + x) for x in exts)

        examples = []
        with open(src_path, encoding='utf-8') as src_file, open(trg_path, encoding='utf-8') as trg_file:
            # src, trg path에서 파일을 불러오고 한줄씩 for문
            for src_line, trg_line in zip(src_file, trg_file):
                src_line, trg_line = src_line.strip(), trg_line.strip() # 오른쪽끝 스페이스 제거.
                # max_length가 있을 경우에는 작업을 해줌.
                if max_length and max_length < max(len(src_line.split()), len(trg_line.split())): 
                    # 스페이스를 띄어쓰기라고 가정, max_len보다 클때(?) 이부분 잘못된거 같은데...
                    '''?????????????????????????????????????????'''
                    continue
                if src_line != '' and trg_line != '':
                    # 별일 없을때 examples에 데이터를 추가.
                    examples += [data.Example.fromlist([src_line, trg_line], fields)]

        super().__init__(examples, fields, **kwargs)


loader = DataLoader(
    config.train,                           # Train file name except extention, which is language.
    config.valid,                           # Validation file name except extension.
    (config.lang[:2], config.lang[-2:]),    # Source and target language. // 예) en, ko -> enko로 등록을 해야함.
    batch_size=config.batch_size,
    device=-1,                              # Lazy loading
    max_length=config.max_length,           # Loger sequence will be excluded.
    dsl=False,                              # Turn-off Dual-supervised Learning mode.
)

# 3.Back to train.py

In [73]:
input_size, output_size = len(loader.src.vocab), len(loader.tgt.vocab)

## 3-1 model

In [76]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack




class Attention(nn.Module):

    def __init__(self, hidden_size):
        super(Attention, self).__init__()

        self.linear = nn.Linear(hidden_size, hidden_size, bias=False) # 맨처음에 projection needed for 가중치 refer to encoder part
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, h_src, h_t_tgt, mask=None):
        # |h_src| = (batch_size, length, hidden_size) - 인코더의 모든 히든 스테잇
        # |h_t_tgt| = (batch_size, 1, hidden_size) - 디코더의 히든 스테잇
        # |mask| = (batch_size, length) - src의 마스킹할 정보

        query = self.linear(h_t_tgt)                     # [B,1,H] * [B,H,H] = [B,1,H]
            # |query| = (batch_size, 1, hidden_size)

        weight = torch.bmm(query, h_src.transpose(1, 2)) # [B,1,H] * [B, H, L] => [B, 1, L] // bmm : batch multiplication
            # |weight| = (batch_size, 1, length)
            
        if mask is not None:
            weight.masked_fill_(mask.unsqueeze(1), -float('inf')) # mask가 있는 부분에 -float('inf')를 넣어줘
        weight = self.softmax(weight)

        context_vector = torch.bmm(weight, h_src)        # [B,1,L]*[B,L,H] -> [B,1,H]
        # |context_vector| = (batch_size, 1, hidden_size)
        # 해석으 해보면, 샘플 데이터에서, 디코더의 시점에서, 어텐션을 적용한 컨텐스트 벡터

        return context_vector


class Encoder(nn.Module):

    def __init__(self, word_vec_size, hidden_size, n_layers=4, dropout_p=.2):
        super(Encoder, self).__init__()

        self.rnn = nn.LSTM(
            word_vec_size, # input shape
            int(hidden_size / 2), # bidirectional 할 것이기 때문에, 나누기 2를 했다. -> 만약 소수점이 되버리면?
            num_layers=n_layers, # stacking LSTM
            dropout=dropout_p,
            bidirectional=True,
            batch_first=True, # batch의 쉐입이 첫번째가 아니라서 앞으로 오게 강제함
        )

    def forward(self, emb):
        # |emb| = (batch_size, length, word_vec_size)

        if isinstance(emb, tuple): # 임베딩 타입이 튜플이니? 
            x, lengths = emb
            x = pack(x, lengths.tolist(), batch_first=True) # https://simonjisu.github.io/nlp/2018/07/05/packedsequence.html
        else:
            x = emb

        y, h = self.rnn(x)

        if isinstance(emb, tuple):
            y, _ = unpack(y, batch_first=True) # 위에 packedsequence가 들어가있으면 풀어줘야 하기 때문에 씀.
        
        # y : [b, n, h]
        # h : [l*2, b, h/2], [l*2, b, h/2]
        return y, h

class Decoder(nn.Module):

    def __init__(self, word_vec_size, hidden_size, n_layers=4, dropout_p=.2):
        super(Decoder, self).__init__()

        # Be aware of value of 'batch_first' parameter and 'bidirectional' parameter.
        self.rnn = nn.LSTM(
            word_vec_size + hidden_size, # input feeding? 을 해줄거기 때문에(concat) 차원이 늘어난다.
            hidden_size,
            num_layers=n_layers,
            dropout=dropout_p,
            bidirectional=False,
            batch_first=True,
        )

    def forward(self, emb_t, h_t_1_tilde, h_t_1):
        '''
        추론할때나, input feeding을 해줄것이기 때문에, 한스텝씩 들어올거야.
        h_t_1_tilde : 저번에 예측한 hidden의 정보값. before softmax
        h_t_1 : h_{t-1} = [h_{t-1}, c_{t-1}]   tuple임. // 전 스텝의 hidden값. //  [n layer, b, h]라는데(?)
        
        # |emb_t| = (b, 1, word_vec_size)
        # |h_t_1_tilde| = (b, 1, h)
        # |h_t_1| = [(n_l, b, h),(n_l, b, h)] : t-1 시점 전의 모든 히든들..같음 not sure
        '''
        batch_size = emb_t.size(0) # [batch]
        hidden_size = h_t_1[0].size(-1) # [hidden]

        if h_t_1_tilde is None:
            # If this is the first time-step, 이제 막 디코더가 시작한것임.
            h_t_1_tilde = emb_t.new(batch_size, 1, hidden_size).zero_() # .new -> 텐서는 디바이스와, 타입이 같아야 arithmetic이 가능한데,.. 그러면 두번을 설정해 줘야함. 귀찮자나..
                                                                                    # 가장 간단하게 하는 방법이. 저 텐서와 같은 디바이스, 타입인놈을 만들어줘. 하는게 new이다.
                                                                        # .zero_() -> inplace 연산이다.

        # Input feeding trick.
        x = torch.cat([emb_t, h_t_1_tilde], dim=-1) # [b, 1, w + h]

        # Unlike encoder, decoder must take an input for sequentially.
        y, h = self.rnn(x, h_t_1) # h_t_1 : [(n_l, b, h), (n_l, b, h)] : 이전 시점의 hidden, context tensors, it is 0 when it's not provided.
            # y : [b, 1, h] // h: [l, b, h],[l,b,h]
            # |decoder_output| = (b, 1, h)
            # |decoder_hidden| = (n, b, h), (n,b,h)
        return y, h


class Generator(nn.Module):

    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()

        self.output = nn.Linear(hidden_size, output_size) # output_size : word vec size
        self.softmax = nn.LogSoftmax(dim=-1) # logSoftmax를 함. (왜?)

    def forward(self, x):
        # |x| = (batch_size, length, hidden_size) : 학습할때는 length길이 만큼 한번에 들어감. 왜냐하면 teacher forcing이니까.

        y = self.softmax(self.output(x)) # linear에 한번 통과한다. 그러면 사이즈가 word sz로 바뀜.
        # |y| = (batch_size, length, output_size)

        # Return log-probability instead of just probability. : 미니배치, 각 샘플별, 각 단어별, 로그 확률값이 리턴이됨.
        return y


class Seq2Seq(nn.Module):

    def __init__(
        self,
        input_size,
        word_vec_size,
        hidden_size,
        output_size,
        n_layers=4,
        dropout_p=.2
    ):

        '''
        input_size : input언어의 vocab size
        word_vec_size : embed size
        hidden_size : hidden sz
        output_size : target언어의 vocab size
        '''

        self.input_size = input_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p

        super(Seq2Seq, self).__init__()


        # 임베드 정의
        self.emb_src = nn.Embedding(input_size, word_vec_size)
        self.emb_dec = nn.Embedding(output_size, word_vec_size)

        # 
        self.encoder = Encoder(
            word_vec_size, hidden_size,
            n_layers=n_layers, dropout_p=dropout_p,
        )
        self.decoder = Decoder(
            word_vec_size, hidden_size,
            n_layers=n_layers, dropout_p=dropout_p,
        )
        self.attn = Attention(hidden_size)

        # attn에서 나온 context vec와 // decoder의 output하고 -> h_tilde
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.tanh = nn.Tanh() # 위 concat에 씌어줄 activation fn
        self.generator = Generator(hidden_size, output_size)

    def generate_mask(self, x, length):
        '''
        x : [bs, n]
        length : [bs,] such as [4,3,1]
        '''
        mask = []

        max_length = max(length)
        for l in length:
            if max_length - l > 0:
                # If the length is shorter than maximum length among samples, 
                # set last few values to be 1s to remove attention weight.
                mask += [torch.cat([x.new_ones(1, l).zero_(),
                                    x.new_ones(1, (max_length - l))
                                    ], dim=-1)]
            else:
                # If the length of the sample equals to maximum length among samples, 
                # set every value in mask to be 0.
                mask += [x.new_ones(1, l).zero_()]

        mask = torch.cat(mask, dim=0).bool() # [[4,4], [4,4], [4,4]] -> [3, 4]짜리 텐서로 flatten

        '''
            length 에) 아래와 같은 텐서가 있을때 

            --- --- --- ---
            |  |   |   |  |  [4,
            ___ ___ ___ ___
            |  |   |   ||||   3,
            --- --- --- ---
            |   ||| ||| |||   1] 라는 x_length모양이 있을것임.
            --- --- --- ---

            --- --- --- ---
            | 0|  0|  0| 0|  
            ___ ___ ___ ___
            | 0|  0|  0| 1|  
            --- --- --- ---
            | 0| 1| | 1| 1|   
            --- --- --- ---
            으로 나오게 한다.
        '''
        return mask



    def merge_encoder_hiddens(self, encoder_hiddens):

        '''
        for loop을 하여 속도가 안좋음.
        '''
        new_hiddens = []
        new_cells = []

        hiddens, cells = encoder_hiddens
            # encoder_hiddens는 hiddens와 cell_state두개를 갖고 있음.
            # hiddens : [2*layers, batch, hidden/2]

        # i-th and (i+1)-th layer is opposite direction.
        # Also, each direction of layer is half hidden size.
        # Therefore, we concatenate both directions to 1 hidden size layer.
        for i in range(0, hiddens.size(0), 2): # 0~2*layers만큼 for문을 돌림.
            new_hiddens += [torch.cat([hiddens[i], hiddens[i + 1]], dim=-1)] # 0,1 // 2,3 // 이런식으로 묶어서 넣어줌.
                # new_hiddens : [bs, hs/2*2] -> [bs, hs]
            new_cells += [torch.cat([cells[i], cells[i + 1]], dim=-1)]

        new_hiddens, new_cells = torch.stack(new_hiddens), torch.stack(new_cells)
                # new_hiddens : [layers, bs, hs]
                # new_cells : [layers, bs, hs]
        return (new_hiddens, new_cells)


    def fast_merge_encoder_hiddens(self, encoder_hiddens):
        '''
        parallel하게 해보자
        encoder : [l*2, b, h/2], [l*2, b, h/2]
        '''
        # Merge bidirectional to uni-directional
        # (layers*2, bs, hs/2) -> (layers, bs, hs).
        # Thus, the converting operation will not working with just 'view' method.
        h_0_tgt, c_0_tgt = encoder_hiddens # 두개 모두 [2layer, b, h/2]
        batch_size = h_0_tgt.size(1)

        # contiguous : 메모리상에 잘 붙어있게 선언하는것.
        # transpose까지 하면 : [b, 2layer, h/2]
        # view : [b, -1, hs] --> [b, layer, h]
        # transpose : [layer, b, h]
        h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                            -1,
                                                            self.hidden_size
                                                            ).transpose(0, 1).contiguous()
        c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                            -1,
                                                            self.hidden_size
                                                            ).transpose(0, 1).contiguous()
        # You can use 'merge_encoder_hiddens' method, instead of using above 3 lines.
        # 'merge_encoder_hiddens' method works with non-parallel way.
        # h_0_tgt = self.merge_encoder_hiddens(h_0_tgt)

        # |h_src| = (batch_size, length, hidden_size)
        # |h_0_tgt| = (n_layers, batch_size, hidden_size)
        # [l, b, h], [l, b, h]
        return h_0_tgt, c_0_tgt

    def forward(self, src, tgt):

        '''
        학습할때는 teacher forcing을 할 것임.

        src : input sentence = [bs, n, V_i]
        tgt : target sentence = [bs, m, V_t]
        '''
        # output = [bs, m, V_t]

        batch_size = tgt.size(0)

        mask = None
        x_length = None
        if isinstance(src, tuple):
            x, x_length = src
            '''
            x_length에서 마스크 정보가 주어지면 generate_mask를 하라고 했음.
            '''
            # Based on the length information, gererate mask to prevent that
            # shorter sample has wasted attention.
            mask = self.generate_mask(x, x_length) 
            # //x : [bs, n] // x_length : [bs,] // mask : [bs, n]
            # |mask| = (batch_size, length)
            '''
            length 에) 아래와 같은 텐서가 있을때 

            --- --- --- ---
            |  |   |   |  |  [4,
            ___ ___ ___ ___
            |  |   |   ||||   3,
            --- --- --- ---
            |   ||| ||| |||   1] 라는 x_length모양이 있을것임.
            --- --- --- ---
            
            즉 [4,3,1]이 들어가 있음. 여기서 배치사이즈는 3임.
            '''

        else:
            x = src

        if isinstance(tgt, tuple):
            tgt = tgt[0]


        # Get word embedding vectors for every time-step of input sentence.
        emb_src = self.emb_src(x) # |emb_src| = (b, n, emb)

        # The last hidden state of the encoder would be a initial hidden state of decoder.
        h_src, h_0_tgt = self.encoder((emb_src, x_length)) # packed_padded_sequence로 처리를 함.
            # |h_src| = (b, n, h) : 인코더의 모든 t시점에서의 히든스테이트
            # |h_0_tgt| = [l*2, b, h/2], [l*2, b, h/2] : 인코더에서 레이어마다 나온 마지막 히든스테이트(컨텍스트)
                # -> 여기서 이친구를 decoder의 init hidden으로 넣어줘야 하는데,feature가 h/2임. 이걸 h로 변환해줘야함.

        h_0_tgt = self.fast_merge_encoder_hiddens(h_0_tgt)
            # merge_encoder_hidden부터 살펴보자
            # [l, b, h], [l, b, h]

        # teacher forcing이기 때문에 정답을 한꺼번에 만들어.
        emb_tgt = self.emb_dec(tgt)
            # |emb_tgt| = (b, l, emb)
        h_tilde = [] # 여기도 한방에 들어갈거야.

        h_t_tilde = None # 첫번째 타임스텝에서는 전에 있던 h_t_tilde는 없다.
        decoder_hidden = h_0_tgt # ([layer, bs, hs], [layer, bs, hs])

        # Run decoder until the end of the time-step.
        for t in range(tgt.size(1)): # length of sentence
            # Teacher Forcing: take each input from training set,
            # not from the last time-step's output.
            # Because of Teacher Forcing,
            # training procedure and inference procedure becomes different.
            # Of course, because of sequential running in decoder,
            # this causes severe bottle-neck.
            emb_t = emb_tgt[:, t, :].unsqueeze(1) # 한 단어씩 번갈아가면서 들어간다. // unsqueeze : 특정 차원에 차원을 추가한다.
                # 인덱싱할 경우 [b, l, emb] -> [b,emb]되버릴 수 있다. 따라서 명시적으로 그냥 선언하자.
            # |emb_t| = (batch_size, 1, word_vec_size)
            # |h_t_tilde| = (batch_size, 1, hidden_size)

            decoder_output, decoder_hidden = self.decoder(emb_t, # 현시점의 단어.
                                                          h_t_tilde, # 지난 타임 스텝의 틸다
                                                          decoder_hidden # [l, b, h], [l, b, h]
                                                          )
            # |decoder_output| = (b, 1, h)
            # |decoder_hidden| = (n, b, h), (n,b,h)


            context_vector = self.attn(h_src, decoder_output, mask)
            # |context_vector| = (batch_size, 1, hidden_size)

            h_t_tilde = self.tanh(self.concat(torch.cat([decoder_output,
                                                         context_vector
                                                         ], dim=-1)))
            # |h_t_tilde| = (batch_size, 1, hidden_size)
            # self.concat -> 2h, h

            h_tilde += [h_t_tilde]

        h_tilde = torch.cat(h_tilde, dim=1)
            # h_tilde = (b, 1, h)
            # concat on dim 1 => (b, m, h)
            # |h_tilde| = (b, length, h)

        y_hat = self.generator(h_tilde)
        # |y_hat| = (b, length, output_size:vocab_size)

        return y_hat


    
    
    
    



def get_model(input_size, output_size, config):
    if config.use_transformer:
        model = Transformer(
            input_size,                     # Source vocabulary size
            config.hidden_size,             # Transformer doesn't need word_vec_size.
            output_size,                    # Target vocabulary size
            n_splits=config.n_splits,       # Number of head in Multi-head Attention.
            n_enc_blocks=config.n_layers,   # Number of encoder blocks
            n_dec_blocks=config.n_layers,   # Number of decoder blocks
            dropout_p=config.dropout,       # Dropout rate on each block
        )
    else:
        model = Seq2Seq(
            input_size,
            config.word_vec_size,           # Word embedding vector size
            config.hidden_size,             # LSTM's hidden vector size
            output_size,
            n_layers=config.n_layers,       # number of layers in LSTM
            dropout_p=config.dropout        # dropout-rate in LSTM
        )

    return model

model = get_model(input_size, output_size, config)

## 3-2 criterion

In [87]:
def get_crit(output_size, pad_index):
    loss_weight = torch.ones(output_size)
        # 패딩이 되어있는곳은 맞춰도 점수를 주고 싶지 않은것.

    loss_weight[pad_index] = 0.
        # 패드인덱스를 받아와서 거기다가 0을 할당해서, 맞춰도 점수를 주지마.
    '''???????????????????????????????????????????????????????????'''
        # pad_index를 가져와야하는데, main함수에서 보면 data_loader의 PAD ; 1을 가져왔으. 왜?
    # Instead of using Cross-Entropy loss,
    # we can use Negative Log-Likelihood(NLL) loss with log-probability.
    crit = nn.NLLLoss(
        weight=loss_weight,
        reduction='sum'
    )

    return crit


crit = get_crit(output_size, 1)

## 3-3 optimizer

In [88]:
def get_optimizer(model, config):
    if config.use_adam:
        if config.use_transformer:
            optimizer = optim.Adam(model.parameters(), lr=config.lr, betas=(.9, .98))
        else: # case of rnn based seq2seq.
            optimizer = optim.Adam(model.parameters(), lr=config.lr)
    elif config.use_radam:
        optimizer = custom_optim.RAdam(model.parameters(), lr=config.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.lr) # initial lr = 1, 그리고 그래디언트가 폭주할거 같으면 클립핑을 함(?)-> 요놈은 한번도 안해봄.

    return optimizer


optimizer = get_optimizer(model, config)

## 3-4 lr_scheduler

In [89]:
def get_scheduler(optimizer, config):
    '''Adam 같은 경우는 쓸 필요가 없고, SGD같은 경우 써야한데'''
    '''
         LR |
          1 |-----------------
            |
            |                  --
            |                      -- ... 10번부터는 lr을 반씩 줄여가면서 진행
            |_____________________________________
               1  2  3  4 ... 9 10 11 12         epochs
    
    '''
    if config.lr_step > 0:
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                                        milestones=[i for i in range(
                                                        max(0, config.lr_decay_start - 1), # 내가 원하는 시작 에폭부터
                                                        (config.init_epoch - 1) + config.n_epochs, # 내가 원하는 끝나는 에폭까지
                                                        config.lr_step # 1
                                                        )],
                                                        gamma=config.lr_gamma, # 0.5씩 작아짐.
                                                        last_epoch=config.init_epoch - 1 if config.init_epoch > 1 else -1,
                                                        )
    else:
        lr_scheduler = None

    return lr_scheduler


lr_scheduler = get_scheduler(optimizer, config)

In [91]:
[i for i in range(max(0, config.lr_decay_start - 1), # 내가 원하는 시작 에폭부터
                (config.init_epoch - 1) + config.n_epochs, # 내가 원하는 끝나는 에폭까지
                config.lr_step # 1
                )]

[9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29]

# 4. trainer.py 

In [107]:
import numpy as np

import torch
from torch import optim
import torch.nn.utils as torch_utils
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler


# Core of the library, contains an engine for training and evaluating, most of the classic machine learning metrics 
# and a variety of handlers to ease the pain of training and validation of neural networks.

from ignite.engine import Engine # Runs a given process_function over each batch of a dataset, emitting events as it goes.
from ignite.engine import Events # Events that are fired by the Engine during execution.
from ignite.metrics import RunningAverage # Compute running average of a metric or the output of process function.
from ignite.contrib.handlers.tqdm_logger import ProgressBar

from decompress.simple_nmt.utils import get_grad_norm, get_parameter_norm


VERBOSE_SILENT = 0
VERBOSE_EPOCH_WISE = 1
VERBOSE_BATCH_WISE = 2

# Gradient init -> FeedForward -> Loss -> back prop -> Gradient descent -> 현제상태 출력 // 을 만들거야. 
# Train, Valid engine두개를 선언하고
# 그 두개를 물고 있는 엔진을 하나 또 설정해
class MaximumLikelihoodEstimationEngine(Engine):

    def __init__(self, func, model, crit, optimizer, lr_scheduler, config):
        self.model = model
        self.crit = crit # criterion (loss)
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.config = config

        super().__init__(func) # 여기서 func은 MaximumLikelihoodEstimationEngine.train임.
        # func을 Engine class에 보내 실행하기.. 근데 그럼 어떤 효과가 잇는거지?
        '''https://pytorch.org/ignite/_modules/ignite/engine/engine.html#Engine'''
        ''' 이해 안되면 이거 해보기.
        class adding():
            def __init__(self, a,b):
                print(a+b)
                return a+b
            
        class bdding(adding):
            def __init__(self):
                z = 1
                super(bdding, self).__init__(1,3)

        bdding()
        '''    
        

        self.best_loss = np.inf
        self.scaler = GradScaler() 


    # static : https://wikidocs.net/21054
    @staticmethod
    #@profile
    def train(engine, mini_batch):
        '''
            engine : 여기에 Engine을 받나본데? // train engine, valid engine
            mini_batch : train_loader가 return하는 객체
                pytorch.DataLoader? 이런 비슷한거 있음.
        '''
        # You have to reset the gradients of all model parameters
        # before to take another step in gradient descent.

        '''
            engine.state.iteration ; Number of iterations the engine has completed. Initialized as 0 and the first iteration is 1.
                https://pytorch.org/ignite/v0.4.6/concepts.html#state
            engine.iteration_per_update : update당 Iteration 숫자... 먼지 모르겟음.
        '''
        # accumulate gradient update
        engine.model.train() # 파라미터 학습 할 것임.
        if engine.state.iteration % engine.config.iteration_per_update == 1 or engine.config.iteration_per_update == 1: # 만약 설정을 config.iteration_per_update 을 1로 했을 경우, 매 iter마다 업데이트, 만약 설정을 10으로 하면 11마다 zero grad함.
            if engine.state.iteration > 1:
                engine.optimizer.zero_grad()

        device = next(engine.model.parameters()).device # 모델이 어느 gpu에 있는지 따오는거
        # 현재 모델이 어느 gpu에 있는지 구해서, 미니배치안에 잇는 텐서들을 해당 디바이스에 보내준다.

        mini_batch.src = (mini_batch.src[0].to(device), mini_batch.src[1]) # encoder tuple, length // 근데 tuple gpu로 옮기는데 length는 안옮기네
        mini_batch.tgt = (mini_batch.tgt[0].to(device), mini_batch.tgt[1]) # decoder x, y
            # tuple은 tensor와 length가 있음.

            # Raw target variable has both BOS and EOS token. 
            # The output of sequence-to-sequence does not have BOS token. 
            # Thus, remove BOS token for reference.
        x, y = mini_batch.src, mini_batch.tgt[0][:, 1:]
            # |x| = (batch_size, length) : x에서는 tensor, length를 가져가고
            # |y| = (batch_size, length) : y(decoder)에서는 length가 빠지고, tensor들만 들어감. 
            # encoder에는 isinstance라는게 들어가서 tuple인지 확인했었어.
            # y의 원래 텐서 모습은 {BOS, y_1, y_2, EOS}이건데, Teacher forcing답안용이기때문에 {y_1, y_2, eos} // 모델에 학습 데이터 들어갈땐 {BOS, y_1, y_2} 들어갈거야.

        with autocast(not engine.config.off_autocast):
                # Take feed-forward
                # Similar as before, the input of decoder does not have EOS token.
                # Thus, remove EOS token for decoder input.
            y_hat = engine.model(x, mini_batch.tgt[0][:, :-1])  
                # 모델에 학습 데이터 들어갈땐 {BOS, y_1, y_2} 들어갈거야.
                # |y_hat| = (b, l, |V|) : 각 미니배치별, length별, 단어별 로그 확률 값이 들어가 있음.

            loss = engine.crit(
                y_hat.contiguous().view(-1, y_hat.size(-1)), # flatten
                y.contiguous().view(-1) # flatten
            )

            backward_target = loss.div(y.size(0)).div(engine.config.iteration_per_update) 
                # NLL을 보면 -1/n *sigma(sgima)인데, Loss정의시 1/n을 안했음. 그래서 직접 나눠줘야함.
                # 그래서 미니 배치 사이즈로 나눠주고, 이터레이션 퍼 업데이트로 나눠줌.

        # gpu가 있을때 autocast가 수행
        if engine.config.gpu_id >= 0 and not engine.config.off_autocast:
            engine.scaler.scale(backward_target).backward()
        else:
            backward_target.backward()

        word_count = int(mini_batch.tgt[1].sum())
            # 단어의 갯수를 알아야 로스를 정확하게 구할 수 있음.
            # 1 : 에는 각 batch별 length가 들어가 있을 것이기 때문에
        p_norm = float(get_parameter_norm(engine.model.parameters())) # parameter norm
        g_norm = float(get_grad_norm(engine.model.parameters())) # gradient norm : 안정적일수록 작아짐.
        '''
                @torch.no_grad()
                def get_grad_norm(parameters, norm_type=2):
                    parameters = list(filter(lambda p: p.grad is not None, parameters)) # 파라미터의수

                    total_norm = 0

                    try:
                        for p in parameters:
                            total_norm += (p.grad.data**norm_type).sum() # ||L2||_norm
                        total_norm = total_norm ** (1. / norm_type) # 
                    except Exception as e:
                        print(e)

                    return total_norm


                @torch.no_grad()
                def get_parameter_norm(parameters, norm_type=2):
                    total_norm = 0

                    try:
                        for p in parameters:
                            total_norm += (p.data**norm_type).sum()
                        total_norm = total_norm ** (1. / norm_type)
                    except Exception as e:
                        print(e)

                    return total_norm
        
        '''


        if engine.state.iteration % engine.config.iteration_per_update == 0 and \
            engine.state.iteration > 0:
                # In order to avoid gradient exploding, we apply gradient clipping.
            torch_utils.clip_grad_norm_(
                engine.model.parameters(),
                engine.config.max_grad_norm,
            )

            # Take a step of gradient descent.
            # GPU가 있을 경우에, scale을 함.
            if engine.config.gpu_id >= 0 and not engine.config.off_autocast:
                # Use scaler instead of engine.optimizer.step() if using GPU.
                engine.scaler.step(engine.optimizer)
                engine.scaler.update()
            else:
                engine.optimizer.step()

            # 만약 lr 스케쥴을 한다그러면..
            # if engine.config.use_noam_decay and engine.lr_scheduler is not None:
            #     engine.lr_scheduler.step()


        loss = float(loss / word_count)
            # 단어단 로스를 구할 수 있음.
        ppl = np.exp(loss)

        return {
            'loss': loss, # 나중에 화면에 출력할 것.
            'ppl': ppl,
            '|param|': p_norm if not np.isnan(p_norm) and not np.isinf(p_norm) else 0., # 학습 초기에 None, inf뜨는 경우가 있었다. 그러면 학습이 안되므로 0을 리턴하도록한다.
            '|g_param|': g_norm if not np.isnan(g_norm) and not np.isinf(g_norm) else 0.,
        }



    
    
    @staticmethod
    def validate(engine, mini_batch):
        engine.model.eval()

        with torch.no_grad():
            device = next(engine.model.parameters()).device
            mini_batch.src = (mini_batch.src[0].to(device), mini_batch.src[1])
            mini_batch.tgt = (mini_batch.tgt[0].to(device), mini_batch.tgt[1])

            x, y = mini_batch.src, mini_batch.tgt[0][:, 1:]
            # |x| = (batch_size, length)
            # |y| = (batch_size, length)

            with autocast(not engine.config.off_autocast):
                y_hat = engine.model(x, mini_batch.tgt[0][:, :-1])
                # |y_hat| = (batch_size, n_classes)
                loss = engine.crit(
                    y_hat.contiguous().view(-1, y_hat.size(-1)),
                    y.contiguous().view(-1),
                )
        
        word_count = int(mini_batch.tgt[1].sum())
        loss = float(loss / word_count)
        ppl = np.exp(loss)

        return {
            'loss': loss,
            'ppl': ppl,
        }




    @staticmethod
    def attach(
        train_engine, validation_engine,
        training_metric_names = ['loss', 'ppl', '|param|', '|g_param|'],
        validation_metric_names = ['loss', 'ppl'],
        verbose=VERBOSE_BATCH_WISE, # 0,1,2 중 하나.
    ):
        # Attaching would be repaeted for serveral metrics.
        # Thus, we can reduce the repeated codes by using this function.
        def attach_running_average(engine, metric_name):
            RunningAverage(output_transform=lambda x: x[metric_name]).attach(
                engine,
                metric_name,
            )

        for metric_name in training_metric_names:
            attach_running_average(train_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120) # progressbar만들어서 성과나오고.
            pbar.attach(train_engine, training_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:

            # 에폭이 끝날때마다 train_engine에서 print안에 있는 친구들을 출력하라고 함.
            @train_engine.on(Events.EPOCH_COMPLETED)
            def print_train_logs(engine):
                avg_p_norm = engine.state.metrics['|param|']
                avg_g_norm = engine.state.metrics['|g_param|']
                avg_loss = engine.state.metrics['loss']

                print('Epoch {} - |param|={:.2e} |g_param|={:.2e} loss={:.4e} ppl={:.2f}'.format(
                    engine.state.epoch,
                    avg_p_norm,
                    avg_g_norm,
                    avg_loss,
                    np.exp(avg_loss),
                ))

        # valid에 대해서도 동일하게 진행
        for metric_name in validation_metric_names:
            attach_running_average(validation_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(validation_engine, validation_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @validation_engine.on(Events.EPOCH_COMPLETED)
            def print_valid_logs(engine):
                avg_loss = engine.state.metrics['loss']

                print('Validation - loss={:.4e} ppl={:.2f} best_loss={:.4e} best_ppl={:.2f}'.format(
                    avg_loss,
                    np.exp(avg_loss),
                    engine.best_loss,
                    np.exp(engine.best_loss),
                ))

    @staticmethod
    def resume_training(engine, resume_epoch):
        engine.state.iteration = (resume_epoch - 1) * len(engine.state.dataloader)
        engine.state.epoch = (resume_epoch - 1)

    @staticmethod
    def check_best(engine):
        loss = float(engine.state.metrics['loss'])
        if loss <= engine.best_loss:
            engine.best_loss = loss

    @staticmethod
    def save_model(engine, train_engine, config, src_vocab, tgt_vocab):
        avg_train_loss = train_engine.state.metrics['loss']
        avg_valid_loss = engine.state.metrics['loss']

        # Set a filename for model of last epoch.
        # We need to put every information to filename, as much as possible.
        model_fn = config.model_fn.split('.')
        
        model_fn = model_fn[:-1] + ['%02d' % train_engine.state.epoch,
                                    '%.2f-%.2f' % (avg_train_loss,
                                                   np.exp(avg_train_loss)
                                                   ),
                                    '%.2f-%.2f' % (avg_valid_loss,
                                                   np.exp(avg_valid_loss)
                                                   )
                                    ] + [model_fn[-1]]

        model_fn = '.'.join(model_fn)

        # Unlike other tasks, we need to save current model, not best model.
        torch.save(
            {
                'model': engine.model.state_dict(),
                'opt': train_engine.optimizer.state_dict(),
                'config': config,
                'src_vocab': src_vocab,
                'tgt_vocab': tgt_vocab,
            }, model_fn
        )
    


In [128]:
vars(Engine(MaximumLikelihoodEstimationEngine.train))

{'_event_handlers': defaultdict(list, {}),
 'logger': <Logger ignite.engine.engine.Engine (WARNING)>,
 '_process_function': <function __main__.MaximumLikelihoodEstimationEngine.train(engine, mini_batch)>,
 'last_event_name': None,
 'should_terminate': False,
 'should_terminate_single_epoch': False,
 'state': State:
 	iteration: 0
 	epoch: 0
 	epoch_length: <class 'NoneType'>
 	max_epochs: <class 'NoneType'>
 	output: <class 'NoneType'>
 	batch: <class 'NoneType'>
 	metrics: <class 'dict'>
 	dataloader: <class 'NoneType'>
 	seed: <class 'NoneType'>
 	times: <class 'dict'>,
 '_state_dict_user_keys': [],
 '_allowed_events': [<Events.EPOCH_STARTED: 'epoch_started'>,
  <Events.EPOCH_COMPLETED: 'epoch_completed'>,
  <Events.STARTED: 'started'>,
  <Events.COMPLETED: 'completed'>,
  <Events.ITERATION_STARTED: 'iteration_started'>,
  <Events.ITERATION_COMPLETED: 'iteration_completed'>,
  <Events.EXCEPTION_RAISED: 'exception_raised'>,
  <Events.GET_BATCH_STARTED: 'get_batch_started'>,
  <Events.

### 4-1 ignite 해부하기

In [108]:
mle_trainer = SingleTrainer(MaximumLikelihoodEstimationEngine, config)

In [114]:
target_engine_class = MaximumLikelihoodEstimationEngine
train_engine = target_engine_class(target_engine_class.train,
                                  model,
                                  crit,
                                  optimizer,
                                  lr_scheduler,
                                  config)

'''
    trainer.py -> MaximumLikelihoodEstimationEngine.train 메서드
                    이 매서드 : Engine(MaximumLikelihoodEstimationEngine.train) 이렇게 되는것임.
                    그리고 기폰 format이  ---- def (engind, batch) ---- 로 정의하는것.임
'''

In [132]:
train_engine.model

Seq2Seq(
  (emb_src): Embedding(1711, 512)
  (emb_dec): Embedding(1421, 512)
  (encoder): Encoder(
    (rnn): LSTM(512, 384, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (rnn): LSTM(1280, 768, num_layers=4, batch_first=True, dropout=0.2)
  )
  (attn): Attention(
    (linear): Linear(in_features=768, out_features=768, bias=False)
    (softmax): Softmax(dim=-1)
  )
  (concat): Linear(in_features=1536, out_features=768, bias=True)
  (tanh): Tanh()
  (generator): Generator(
    (output): Linear(in_features=768, out_features=1421, bias=True)
    (softmax): LogSoftmax(dim=-1)
  )
)

In [133]:
train_engine.state

State:
	iteration: 0
	epoch: 0
	epoch_length: <class 'NoneType'>
	max_epochs: <class 'NoneType'>
	output: <class 'NoneType'>
	batch: <class 'NoneType'>
	metrics: <class 'dict'>
	dataloader: <class 'NoneType'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [135]:
vars(train_engine.config)


{'is_continue': False,
 'model_fn': '.',
 'train': '/Users/rainism/desktop/grad/torch_study/transformer/data/tt.corpus.shuf.train.tok.bpe',
 'valid': '/Users/rainism/desktop/grad/torch_study/transformer/data/tt.corpus.shuf.train.tok.bpe',
 'lang': 'enko',
 'gpu_id': -1,
 'store_true': False,
 'batch_size': 160,
 'n_epochs': 30,
 'verbose': 1,
 'init_epoch': 1,
 'max_length': 100,
 'dropout': 0.2,
 'word_vec_size': 512,
 'hidden_size': 768,
 'n_layers': 4,
 'max_grad_norm': 5.0,
 'iteration_per_update': 1,
 'lr': 1.0,
 'lr_step': 1,
 'lr_gamma': 0.5,
 'lr_decay_start': 10,
 'use_adam': True,
 'rl_lr': 0.01,
 'rl_n_samples': 1,
 'rl_n_epochs': 10,
 'rl_n_gram': 6,
 'rl_reward': 'gleu',
 'use_transformer': False,
 'n_splits': 8}

In [137]:
train_engine.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 1.0
    lr: 1.0
    weight_decay: 0
)

In [140]:
next(train_engine.model.parameters()).device

device(type='cpu')

In [143]:
next(iter(train_iter)).src

(tensor([[ 292,   11, 1603,  ...,  117,  140,    4],
         [  23,   97,    5,  ...,    1,    1,    1],
         [  23, 1373,   78,  ...,    1,    1,    1],
         ...,
         [  42,   18,  318,  ...,    1,    1,    1],
         [1171, 1236,  346,  ...,    1,    1,    1],
         [ 304,  203,    8,  ...,    1,    1,    1]]),
 tensor([90, 86, 83, 67, 66, 66, 64, 61, 59, 58, 57, 54, 54, 53, 53, 53, 53, 51,
         51, 51, 51, 50, 49, 49, 49, 48, 47, 47, 46, 45, 44, 44, 42, 42, 41, 41,
         40, 39, 39, 37, 37, 36, 36, 36, 34, 33, 33, 33, 32, 32, 31, 31, 31, 30,
         29, 29, 28, 27, 26, 25, 25, 24, 23, 23, 22, 21, 21, 21, 20, 20, 20, 20,
         19, 18, 18, 17, 17, 16, 16, 15, 15, 15, 15, 14, 14, 14, 13, 13, 13, 12,
         12, 12, 11, 11, 11,  9,  9,  8,  6,  6]))

In [146]:
next(iter(train_iter)).tgt[0][:,1:]

tensor([[  96,  107,  214,  ...,    1,    1,    1],
        [ 155,   11,  873,  ...,    1,    1,    1],
        [ 318,  396,   19,  ..., 1178,    4,    3],
        ...,
        [  47, 1228,  115,  ...,    1,    1,    1],
        [  71,  838,  933,  ...,    1,    1,    1],
        [ 114,  217, 1115,  ...,    1,    1,    1]])

In [150]:
autocast

torch.cuda.amp.autocast_mode.autocast

In [170]:
train_engine.crit

NLLLoss()

In [153]:
x,y = next(iter(train_iter)).src, next(iter(train_iter)).tgt[0][:,1:]

In [159]:
y_hat = train_engine.model(x, next(iter(train_iter)).tgt[0][:,:-1])

In [161]:
y_hat.shape

torch.Size([100, 76, 1421])

In [163]:
loss = train_engine.crit(y_hat.contiguous().view(-1, y_hat.size(-1)), y.contiguous().view(-1))

In [199]:
backward_target = loss.div(y.size(0)).div(train_engine.config.iteration_per_update)

In [167]:
next(iter(train_iter)).tgt[1]

tensor([60, 65, 77, 44, 59, 46, 62, 49, 45, 62, 49, 45, 45, 59, 56, 51, 48, 58,
        52, 47, 46, 46, 56, 48, 44, 50, 51, 42, 52, 36, 55, 40, 49, 45, 46, 42,
        42, 52, 44, 37, 34, 37, 37, 35, 44, 40, 39, 37, 50, 37, 36, 35, 29, 38,
        36, 26, 31, 27, 36, 34, 25, 22, 24, 21, 27, 26, 25, 21, 27, 21, 19, 19,
        26, 30, 22, 25, 25, 24, 18, 23, 22, 18, 16, 22, 17, 16, 20, 18,  9, 20,
        19, 17, 16, 14, 12, 16, 14, 15, 13,  9])

In [202]:
# float16 사용하려고함. 한번 뤱핑해주는거

train_engine.scaler.scale(backward_target).backward()

In [205]:
word_count = int(next(iter(train_iter)).tgt[1].sum())
loss = float(loss/word_count)
ppl = np.exp(loss)

### 4-2 validation enigne

In [171]:
validation_engine = target_engine_class(target_engine_class.validate,
                                  model,
                                  crit,
                                  optimizer,
                                  lr_scheduler,
                                  config)

/Users/rainism/miniforge3/envs/tensorflow/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


### 4-3 attatch

In [195]:
training_metric_names = ['loss', 'ppl', '|param|', '|g_param|'],

def attach_running_average(engine, metrix_name):
    RunningAverage(output_transform = lambda x: x[metric_name]).attach(engine, metric_name)
    
for metric_name in training_metric_names:
    attach_running_average(train_engine, metric_name)


In [196]:
train_engine.state.metrics

{}

In [177]:
target_engine_class.attach(train_engine, validation_engine, verbose = config.verbose)

In [179]:
vars(target_engine_class)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.MaximumLikelihoodEstimationEngine.__init__(self, func, model, crit, optimizer, lr_scheduler, config)>,
              'train': <staticmethod at 0x146aec880>,
              'validate': <staticmethod at 0x146aecac0>,
              'attach': <staticmethod at 0x146aece80>,
              'resume_training': <staticmethod at 0x146aec370>,
              'check_best': <staticmethod at 0x146aecee0>,
              'save_model': <staticmethod at 0x1471150d0>,
              '__doc__': None})

In [185]:
vars(train_engine)

{'model': Seq2Seq(
   (emb_src): Embedding(1711, 512)
   (emb_dec): Embedding(1421, 512)
   (encoder): Encoder(
     (rnn): LSTM(512, 384, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
   )
   (decoder): Decoder(
     (rnn): LSTM(1280, 768, num_layers=4, batch_first=True, dropout=0.2)
   )
   (attn): Attention(
     (linear): Linear(in_features=768, out_features=768, bias=False)
     (softmax): Softmax(dim=-1)
   )
   (concat): Linear(in_features=1536, out_features=768, bias=True)
   (tanh): Tanh()
   (generator): Generator(
     (output): Linear(in_features=768, out_features=1421, bias=True)
     (softmax): LogSoftmax(dim=-1)
   )
 ),
 'crit': NLLLoss(),
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     initial_lr: 1.0
     lr: 1.0
     weight_decay: 0
 ),
 'lr_scheduler': <torch.optim.lr_scheduler.MultiStepLR at 0x146da7160>,
 'config': Namespace(batch_size=160, dropout=0.2, gpu_id=-1, hidden_size=768, init_ep

In [231]:
loader.src.numericalize

<bound method Field.numericalize of <torchtext.data.field.Field object at 0x1473e77f0>>

In [225]:
lengths = [1,2,3,4,5]
original_indice = [5,4,3,2,1]
lines = torch.randn(5,5,10)

In [226]:
from operator import itemgetter

sorted(zip(lines, lengths, original_indice), key = itemgetter(1), reverse=True)

[(tensor([[ 0.4472,  0.3944, -0.0200,  0.3257, -1.0150,  1.1418, -1.7212,  1.0884,
            0.4792,  0.4483],
          [ 1.9648,  0.1412, -0.7832, -0.6083, -0.4342, -0.4592,  2.2842, -1.3607,
            0.7157, -0.0124],
          [-0.8316, -0.4368,  1.0706, -1.1156,  0.3405,  0.1276,  0.7088, -1.4126,
           -1.0587, -0.6145],
          [-0.9105,  0.1868,  0.4701, -0.6926, -0.4354,  1.1676, -2.5950, -0.1424,
            0.1556, -1.3227],
          [-0.3083,  1.0997,  0.9196,  0.9776,  0.3829,  0.7150,  1.1635, -1.0842,
           -0.7882,  0.6775]]),
  5,
  1),
 (tensor([[-4.2061e-01, -1.1096e+00,  1.1334e+00,  1.0778e+00, -3.8335e-01,
            2.3949e-01,  6.2579e-01, -1.7524e-01, -3.3979e-04, -9.2623e-01],
          [-1.0018e+00,  9.2463e-01, -1.7597e+00,  6.6523e-01, -7.4228e-01,
           -8.4601e-02,  1.1326e+00,  1.0051e+00, -1.5662e-01, -1.0116e-01],
          [-8.0829e-01,  8.4494e-01,  2.1979e+00,  1.5576e+00,  4.8203e-01,
            4.8864e-01, -1.0841e+00,  5.

In [220]:
lengths

[1, 2, 3, 4, 5]